In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime


In [2]:
from test_key import g_key
import gmaps
import os
import matplotlib.pyplot as plt
from test_key import bls_api_key


## Demographic unemployment rate

In [3]:
# Pull gender, race, educational attainment, and age data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LNS14000000", "LNS14000001", "LNS14000002", "LNS14000003", "LNS14000006", "LNS14032183", "LNS14000009",
         "LNS14027659", "LNS14027660", "LNS14027689", "LNS14027662", "LNS14024887", "LNS14000089", "LNS14000091",
                "LNS14000093", "LNS14024230"]

start_year = "2008"
end_year = "2011"

parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data = json.loads(p.text)

json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 494,
 'message': [],
 'Results': {'series': [{'seriesID': 'LNS14000000',
    'data': [{'year': '2011',
      'period': 'M12',
      'periodName': 'December',
      'value': '8.5',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M11',
      'periodName': 'November',
      'value': '8.6',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M10',
      'periodName': 'October',
      'value': '8.8',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M09',
      'periodName': 'September',
      'value': '9.0',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M08',
      'periodName': 'August',
      'value': '9.0',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M07',
      'periodName': 'July',
      'value': '9.0',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M06',
      'periodName': 'June',
      'value': '9.1',
      'footnotes': [{}]},
     

In [4]:
years = []
months = []
date = []

for data_point in np.arange(len(json_data["Results"]["series"][0]["data"])):
        years.append(json_data["Results"]["series"][0]["data"][data_point]["year"])
        months.append(json_data["Results"]["series"][0]["data"][data_point]["periodName"])
        date.append(f"{json_data['Results']['series'][0]['data'][data_point]['year']}-{json_data['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [5]:
# Create DataFrame

unemployment_df = pd.DataFrame({"Date": date,
                                "Year": years,
                                "Month": months,
                                "LNS14000000" : "",
                                "LNS14000001" : "",
                                "LNS14000002" : "",
                                "LNS14000003" : "",
                                "LNS14000006" : "",
                                "LNS14032183" : "",
                                "LNS14000009" : "",
                                "LNS14027659" : "",
                                "LNS14027660" : "",
                                "LNS14027689" : "",
                                "LNS14027662" : "",
                                "LNS14024887" : "",
                                "LNS14000089" : "",
                                "LNS14000091" : "",
                                "LNS14000093" : "",
                                "LNS14024230" : ""})
unemployment_df.head()

,Date,Year,Month,LNS14000000,LNS14000001,LNS14000002,LNS14000003,LNS14000006,LNS14032183,LNS14000009,LNS14027659,LNS14027660,LNS14027689,LNS14027662,LNS14024887,LNS14000089,LNS14000091,LNS14000093,LNS14024230
0,2011-12,2011,December,,,,,,,,,,,,,,,,
1,2011-11,2011,November,,,,,,,,,,,,,,,,
2,2011-10,2011,October,,,,,,,,,,,,,,,,
3,2011-09,2011,September,,,,,,,,,,,,,,,,
4,2011-08,2011,August,,,,,,,,,,,,,,,,


In [6]:
# Fills in values

for series in np.arange(len(json_data["Results"]["series"])):
    for data_point in np.arange(len(json_data["Results"]["series"][series]["data"])):
        unemployment_df.loc[data_point, json_data["Results"]["series"][series]["seriesID"]] = json_data["Results"]["series"][series]["data"][data_point]["value"]

In [7]:
# Rename column labels

unemployment_df.rename(columns={"LNS14000000": "Unemployment Rate for 16yo and over",
                                "LNS14000001": "Unemployment Rate for Men 16yo and over",
                                "LNS14000002": "Unemployment Rate for Women 16yo and over",
                                "LNS14000003": "Unemployment Rate for White people 16yo and over",
                                "LNS14000006": "Unemployment Rate for African Americans 16yo and over",
                                "LNS14032183": "Unemployment Rate for Asian people 16yo and over",
                                "LNS14000009": "Unemployment Rate for Hispanic people 16yo and over",
                                "LNS14027659": "Unemployment Rate for 25yo and older with less than High School Diploma",
                                "LNS14027660": "Unemployment Rate for 25yo and older with High School Degree",
                                "LNS14027689": "Unemployment Rate for 25yo and older with some college",
                                "LNS14027662": "Unemployment Rate for 25yo and older with Bachelor's degree or higher",
                                "LNS14024887": "Unemployment Rate for 16-24yo",
                                "LNS14000089": "Unemployment Rate for 25-34yo",
                                "LNS14000091": "Unemployment Rate for 35-44yo",
                                "LNS14000093": "Unemployment Rate for 45-54yo",
                                "LNS14024230": "Unemployment Rate for 55yo and over"},
                      inplace=True)

unemployment_df.sort_values(by="Date", ascending=True, inplace=True)
unemployment_df.to_csv("Unemployment_Rates.csv", index=False)

unemployment_df

,Date,Year,Month,Unemployment Rate for 16yo and over,Unemployment Rate for Men 16yo and over,Unemployment Rate for Women 16yo and over,Unemployment Rate for White people 16yo and over,Unemployment Rate for African Americans 16yo and over,Unemployment Rate for Asian people 16yo and over,Unemployment Rate for Hispanic people 16yo and over,Unemployment Rate for 25yo and older with less than High School Diploma,Unemployment Rate for 25yo and older with High School Degree,Unemployment Rate for 25yo and older with some college,Unemployment Rate for 25yo and older with Bachelor's degree or higher,Unemployment Rate for 16-24yo,Unemployment Rate for 25-34yo,Unemployment Rate for 35-44yo,Unemployment Rate for 45-54yo,Unemployment Rate for 55yo and over
47,2008-01,2008,January,5.0,5.2,4.8,4.4,9.1,3.1,6.3,7.7,4.7,3.7,2.1,11.7,5.1,3.6,3.4,3.3
46,2008-02,2008,February,4.9,5.0,4.7,4.4,8.4,2.9,6.2,7.4,4.7,3.8,2.0,11.4,4.9,3.6,3.4,3.3
45,2008-03,2008,March,5.1,5.2,5.0,4.5,9.2,3.6,6.9,8.4,5.1,3.9,2.1,11.4,5.3,3.8,3.5,3.4
44,2008-04,2008,April,5.0,5.2,4.8,4.4,8.6,3.5,7.1,7.7,5.0,4.0,2.1,11.0,5.1,3.9,3.7,3.0
43,2008-05,2008,May,5.4,5.6,5.2,4.8,9.6,4.1,6.9,8.1,5.0,4.3,2.2,13.0,5.2,4.2,3.8,3.2
42,2008-06,2008,June,5.6,5.8,5.3,5.0,9.4,4.2,7.6,8.7,5.1,4.3,2.4,12.9,5.4,4.4,3.8,3.3
41,2008-07,2008,July,5.8,6.2,5.3,5.2,10.0,3.7,7.5,8.6,5.4,4.6,2.5,13.5,5.7,4.7,3.8,3.7
40,2008-08,2008,August,6.1,6.3,5.9,5.4,10.6,4.5,8.0,9.7,5.8,4.9,2.8,13.1,6.2,5.1,4.3,4.2
39,2008-09,2008,September,6.1,6.7,5.5,5.4,11.3,4.0,8.0,9.8,6.2,5.0,2.6,13.5,6.2,5.1,4.3,4.2
38,2008-10,2008,October,6.5,7.1,5.9,5.9,11.4,3.9,8.8,10.3,6.4,5.2,3.1,13.6,6.7,5.4,4.6,4.5


## Statewide unemployment rate

In [8]:
# Pull state data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LASST010000000000003", "LASST020000000000003", "LASST040000000000003", "LASST050000000000003",
                 "LASST060000000000003", "LASST080000000000003", "LASST090000000000003", "LASST100000000000003",
                 "LASST110000000000003", "LASST120000000000003", "LASST130000000000003", "LASST150000000000003",
                 "LASST160000000000003", "LASST170000000000003", "LASST180000000000003", "LASST190000000000003",
                 "LASST200000000000003", "LASST210000000000003", "LASST220000000000003", "LASST230000000000003",
                 "LASST240000000000003", "LASST250000000000003", "LASST260000000000003", "LASST270000000000003",
                 "LASST280000000000003", "LASST290000000000003", "LASST300000000000003", "LASST310000000000003",
                 "LASST320000000000003", "LASST330000000000003", "LASST340000000000003", "LASST350000000000003",
                 "LASST360000000000003", "LASST370000000000003", "LASST380000000000003", "LASST390000000000003",
                 "LASST400000000000003", "LASST410000000000003", "LASST420000000000003", "LASST720000000000003",
                 "LASST440000000000003", "LASST450000000000003", "LASST460000000000003", "LASST470000000000003",
                 "LASST480000000000003", "LASST490000000000003", "LASST500000000000003", "LASST510000000000003",
                 "LASST530000000000003", "LASST540000000000003"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)
json_data2 = json.loads(p.text)


json_data2

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 1294,
 'message': [],
 'Results': {'series': [{'seriesID': 'LASST010000000000003',
    'data': [{'year': '2011',
      'period': 'M12',
      'periodName': 'December',
      'value': '8.3',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M11',
      'periodName': 'November',
      'value': '8.6',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M10',
      'periodName': 'October',
      'value': '9.0',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M09',
      'periodName': 'September',
      'value': '9.4',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M08',
      'periodName': 'August',
      'value': '9.7',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M07',
      'periodName': 'July',
      'value': '9.9',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M06',
      'periodName': 'June',
      'value': '10.0',
      'footnotes': [

In [10]:
# Pull state data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LASST550000000000003", "LASST560000000000003"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data3 = json.loads(p.text)

json_data3

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 128,
 'message': [],
 'Results': {'series': [{'seriesID': 'LASST550000000000003',
    'data': [{'year': '2011',
      'period': 'M12',
      'periodName': 'December',
      'value': '7.3',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M11',
      'periodName': 'November',
      'value': '7.4',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M10',
      'periodName': 'October',
      'value': '7.5',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M09',
      'periodName': 'September',
      'value': '7.7',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M08',
      'periodName': 'August',
      'value': '7.8',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M07',
      'periodName': 'July',
      'value': '7.8',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M06',
      'periodName': 'June',
      'value': '7.9',
      'footnotes': [{}

In [11]:
years = []
months = []
date = []

for data_point in np.arange(len(json_data2["Results"]["series"][0]["data"])):
        years.append(json_data2["Results"]["series"][0]["data"][data_point]["year"])
        months.append(json_data2["Results"]["series"][0]["data"][data_point]["periodName"])
        date.append(f"{json_data['Results']['series'][0]['data'][data_point]['year']}-{json_data['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [12]:
# Create DataFrame

state_unemployment_df = pd.DataFrame({"Date": date,
                                "Year": years,
                                "Month": months,
                                "LASST010000000000003": "", "LASST020000000000003": "", "LASST040000000000003": "",
                                "LASST050000000000003": "", "LASST060000000000003": "", "LASST080000000000003": "",
                                "LASST090000000000003": "", "LASST100000000000003": "", "LASST110000000000003": "",
                                "LASST120000000000003": "", "LASST130000000000003": "", "LASST150000000000003": "",
                                "LASST160000000000003": "", "LASST170000000000003": "", "LASST180000000000003": "",
                                "LASST190000000000003": "", "LASST200000000000003": "", "LASST210000000000003": "",
                                "LASST220000000000003": "", "LASST230000000000003": "", "LASST240000000000003": "",
                                "LASST250000000000003": "", "LASST260000000000003": "", "LASST270000000000003": "",
                                "LASST280000000000003": "", "LASST290000000000003": "", "LASST300000000000003": "",
                                "LASST310000000000003": "", "LASST320000000000003": "", "LASST330000000000003": "",
                                "LASST340000000000003": "", "LASST350000000000003": "", "LASST360000000000003": "",
                                "LASST370000000000003": "", "LASST380000000000003": "", "LASST390000000000003": "",
                                "LASST400000000000003": "", "LASST410000000000003": "", "LASST420000000000003": "",
                                "LASST720000000000003": "", "LASST440000000000003": "", "LASST450000000000003": "",
                                "LASST460000000000003": "", "LASST470000000000003": "", "LASST480000000000003": "",
                                "LASST490000000000003": "", "LASST500000000000003": "", "LASST510000000000003": "",
                                "LASST530000000000003": "", "LASST540000000000003": "", "LASST550000000000003": "",
                                "LASST560000000000003": ""})
state_unemployment_df.head()
state_unemployment_df.head()

,Date,Year,Month,LASST010000000000003,LASST020000000000003,LASST040000000000003,LASST050000000000003,LASST060000000000003,LASST080000000000003,LASST090000000000003,...,LASST460000000000003,LASST470000000000003,LASST480000000000003,LASST490000000000003,LASST500000000000003,LASST510000000000003,LASST530000000000003,LASST540000000000003,LASST550000000000003,LASST560000000000003
0,2011-12,2011,December,,,,,,,,...,,,,,,,,,,
1,2011-11,2011,November,,,,,,,,...,,,,,,,,,,
2,2011-10,2011,October,,,,,,,,...,,,,,,,,,,
3,2011-09,2011,September,,,,,,,,...,,,,,,,,,,
4,2011-08,2011,August,,,,,,,,...,,,,,,,,,,


In [13]:

state_unemployment_df = pd.DataFrame({"Date": date,
                                      "Year": years,
                                      
                                      
                                "LASST010000000000003": "", "LASST020000000000003": "", "LASST040000000000003": "",
                                "LASST050000000000003": "", "LASST060000000000003": "", "LASST080000000000003": "",
                                "LASST090000000000003": "", "LASST100000000000003": "", "LASST110000000000003": "",
                                "LASST120000000000003": "", "LASST130000000000003": "", "LASST150000000000003": "",
                                "LASST160000000000003": "", "LASST170000000000003": "", "LASST180000000000003": "",
                                "LASST190000000000003": "", "LASST200000000000003": "", "LASST210000000000003": "",
                                "LASST220000000000003": "", "LASST230000000000003": "", "LASST240000000000003": "",
                                "LASST250000000000003": "", "LASST260000000000003": "", "LASST270000000000003": "",
                                "LASST280000000000003": "", "LASST290000000000003": "", "LASST300000000000003": "",
                                "LASST310000000000003": "", "LASST320000000000003": "", "LASST330000000000003": "",
                                "LASST340000000000003": "", "LASST350000000000003": "", "LASST360000000000003": "",
                                "LASST370000000000003": "", "LASST380000000000003": "", "LASST390000000000003": "",
                                "LASST400000000000003": "", "LASST410000000000003": "", "LASST420000000000003": "",
                                "LASST720000000000003": "", "LASST440000000000003": "", "LASST450000000000003": "",
                                "LASST460000000000003": "", "LASST470000000000003": "", "LASST480000000000003": "",
                                "LASST490000000000003": "", "LASST500000000000003": "", "LASST510000000000003": "",
                                "LASST530000000000003": "", "LASST540000000000003": "", "LASST550000000000003": "",
                                "LASST560000000000003": ""})
state_unemployment_df.head()

,Date,Year,LASST010000000000003,LASST020000000000003,LASST040000000000003,LASST050000000000003,LASST060000000000003,LASST080000000000003,LASST090000000000003,LASST100000000000003,...,LASST460000000000003,LASST470000000000003,LASST480000000000003,LASST490000000000003,LASST500000000000003,LASST510000000000003,LASST530000000000003,LASST540000000000003,LASST550000000000003,LASST560000000000003
0,2011-12,2011,,,,,,,,,...,,,,,,,,,,
1,2011-11,2011,,,,,,,,,...,,,,,,,,,,
2,2011-10,2011,,,,,,,,,...,,,,,,,,,,
3,2011-09,2011,,,,,,,,,...,,,,,,,,,,
4,2011-08,2011,,,,,,,,,...,,,,,,,,,,


In [14]:
# Fills in values

for series2 in np.arange(len(json_data2["Results"]["series"])):
    for data_point2 in np.arange(len(json_data2["Results"]["series"][series2]["data"])):
        state_unemployment_df.loc[data_point2, json_data2["Results"]["series"][series2]["seriesID"]] = json_data2["Results"]["series"][series2]["data"][data_point2]["value"]

for series3 in np.arange(len(json_data3["Results"]["series"])):
    for data_point3 in np.arange(len(json_data3["Results"]["series"][series3]["data"])):
        state_unemployment_df.loc[data_point3, json_data3["Results"]["series"][series3]["seriesID"]] = json_data3["Results"]["series"][series3]["data"][data_point3]["value"]

In [15]:
# Rename Column labels

state_unemployment_df.rename(columns={"LASST010000000000003" : "Alabama", "LASST020000000000003" : "Alaska",
                                      "LASST040000000000003" : "Arizona", "LASST050000000000003" : "Arkansas",
                                      "LASST060000000000003" : "California", "LASST080000000000003" : "Colorado",
                                      "LASST090000000000003" : "Connecticut", "LASST100000000000003" : "Delaware",
                                      "LASST110000000000003" : "District Of Columbia", "LASST120000000000003" : "Florida",
                                      "LASST130000000000003" : "Georgia", "LASST150000000000003" : "Hawaii",
                                      "LASST160000000000003" : "Idaho", "LASST170000000000003" : "Illinois",
                                      "LASST180000000000003" : "Indiana", "LASST190000000000003" : "Iowa",
                                      "LASST200000000000003" : "Kansas", "LASST210000000000003" : "Kentucky",
                                      "LASST220000000000003" : "Louisiana", "LASST230000000000003" : "Maine",
                                      "LASST240000000000003" : "Maryland", "LASST250000000000003" : "Massachusetts",
                                      "LASST260000000000003" : "Michigan", "LASST270000000000003" : "Minnesota",
                                      "LASST280000000000003" : "Mississippi", "LASST290000000000003" : "Missouri",
                                      "LASST300000000000003" : "Montana", "LASST310000000000003" : "Nebraska",
                                      "LASST320000000000003" : "Nevada", "LASST330000000000003" : "New Hampshire",
                                      "LASST340000000000003" : "New Jersey", "LASST350000000000003" : "New Mexico",
                                      "LASST360000000000003" : "New York", "LASST370000000000003" : "North Carolina",
                                      "LASST380000000000003" : "North Dakota", "LASST390000000000003" : "Ohio",
                                      "LASST400000000000003" : "Oklahoma", "LASST410000000000003" : "Oregon",
                                      "LASST420000000000003" : "Pennsylvania", "LASST720000000000003" : "Puerto Rico",
                                      "LASST440000000000003" : "Rhode Island", "LASST450000000000003" : "South Carolina",
                                      "LASST460000000000003" : "South Dakota", "LASST470000000000003" : "Tennessee",
                                      "LASST480000000000003" : "Texas", "LASST490000000000003" : "Utah",
                                      "LASST500000000000003" : "Vermont", "LASST510000000000003" : "Virginia",
                                      "LASST530000000000003" : "Washington", "LASST540000000000003" : "West Virginia",
                                      "LASST550000000000003" : "Wisconsin", "LASST560000000000003" : "Wyoming"},
                             inplace=True)

state_unemployment_df.sort_values(by="Date", ascending=True, inplace=True)

state_unemployment_df.to_csv("State_Unemployment_Rates.csv", index=False)

state_unemployment_df

,Date,Year,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
47,2008-01,2008,4.4,6.5,4.5,5.0,6.0,4.1,4.9,3.8,...,2.6,5.4,4.3,3.0,4.3,3.4,4.7,4.5,4.5,2.8
46,2008-02,2008,4.5,6.5,4.7,5.0,6.1,4.2,5.0,3.9,...,2.6,5.5,4.3,3.0,4.3,3.4,4.7,4.4,4.4,2.8
45,2008-03,2008,4.6,6.6,5.0,4.9,6.2,4.3,5.1,4.0,...,2.6,5.6,4.3,3.1,4.3,3.4,4.8,4.3,4.3,2.8
44,2008-04,2008,4.8,6.6,5.3,5.0,6.5,4.4,5.2,4.1,...,2.7,5.8,4.4,3.2,4.4,3.5,4.9,4.2,4.3,2.8
43,2008-05,2008,5.1,6.7,5.6,5.0,6.7,4.5,5.3,4.3,...,2.8,6.1,4.5,3.2,4.4,3.7,5.0,4.1,4.4,2.8
42,2008-06,2008,5.3,6.7,5.9,5.2,7.0,4.7,5.5,4.6,...,2.9,6.4,4.6,3.3,4.5,3.8,5.2,4.1,4.5,2.9
41,2008-07,2008,5.6,6.7,6.3,5.3,7.3,4.9,5.7,4.9,...,3.1,6.7,4.8,3.4,4.6,3.9,5.4,4.0,4.7,2.9
40,2008-08,2008,5.9,6.7,6.6,5.5,7.6,5.0,5.9,5.2,...,3.2,6.9,4.9,3.6,4.7,4.0,5.5,4.1,4.9,3.1
39,2008-09,2008,6.2,6.8,6.9,5.8,7.9,5.2,6.1,5.6,...,3.4,7.2,5.1,3.7,4.8,4.2,5.7,4.2,5.2,3.2
38,2008-10,2008,6.7,6.8,7.3,6.1,8.3,5.4,6.3,6.0,...,3.5,7.5,5.3,3.9,5.1,4.4,6.0,4.4,5.5,3.3


In [23]:

ystate_unemployment_df = state_unemployment_df.groupby("Year").astype(float).mean()
yystate_unemployment_df =state_unemployment_df.transpose()
states_df = pd.read_csv("Resources/states.csv", delimiter="\t")
ystate_unemployment_df = pd.merge(state_unemployment_df,states_df, on="name", how="inner")
ystate_unemployment_df = state_unemployment_df.drop(["state"],axis=1)
ystate_unemployment_df.to_csv("state_unemployment_rate.csv")

AttributeError: Cannot access callable attribute 'astype' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [24]:
t_df = state_unemployment_df.transpose(copy=True)
t_df.head()

,47,46,45,44,43,42,41,40,39,38,...,9,8,7,6,5,4,3,2,1,0
Year,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,...,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
Alabama,4.4,4.5,4.6,4.8,5.1,5.3,5.6,5.9,6.2,6.7,...,10.1,10.0,10.0,10.0,9.9,9.7,9.4,9.0,8.6,8.3
Alaska,6.5,6.5,6.6,6.6,6.7,6.7,6.7,6.7,6.8,6.8,...,7.7,7.6,7.6,7.5,7.5,7.5,7.5,7.5,7.5,7.4
Arizona,4.5,4.7,5.0,5.3,5.6,5.9,6.3,6.6,6.9,7.3,...,9.7,9.7,9.7,9.7,9.7,9.6,9.4,9.2,9.0,8.8
Arkansas,5.0,5.0,4.9,5.0,5.0,5.2,5.3,5.5,5.8,6.1,...,8.5,8.5,8.5,8.4,8.4,8.3,8.2,8.0,7.9,7.8


In [25]:
t_df = state_unemployment_df.transpose(copy=True)
t_df.head()

,47,46,45,44,43,42,41,40,39,38,...,9,8,7,6,5,4,3,2,1,0
Year,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,...,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
Alabama,4.4,4.5,4.6,4.8,5.1,5.3,5.6,5.9,6.2,6.7,...,10.1,10.0,10.0,10.0,9.9,9.7,9.4,9.0,8.6,8.3
Alaska,6.5,6.5,6.6,6.6,6.7,6.7,6.7,6.7,6.8,6.8,...,7.7,7.6,7.6,7.5,7.5,7.5,7.5,7.5,7.5,7.4
Arizona,4.5,4.7,5.0,5.3,5.6,5.9,6.3,6.6,6.9,7.3,...,9.7,9.7,9.7,9.7,9.7,9.6,9.4,9.2,9.0,8.8
Arkansas,5.0,5.0,4.9,5.0,5.0,5.2,5.3,5.5,5.8,6.1,...,8.5,8.5,8.5,8.4,8.4,8.3,8.2,8.0,7.9,7.8


In [26]:
t_df.columns = t_df.loc["Year"].to_numpy()
t_df.head()

,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,...,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
Year,2008,2008,2008,2008,2008,2008,2008,2008,2008,2008,...,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
Alabama,4.4,4.5,4.6,4.8,5.1,5.3,5.6,5.9,6.2,6.7,...,10.1,10.0,10.0,10.0,9.9,9.7,9.4,9.0,8.6,8.3
Alaska,6.5,6.5,6.6,6.6,6.7,6.7,6.7,6.7,6.8,6.8,...,7.7,7.6,7.6,7.5,7.5,7.5,7.5,7.5,7.5,7.4
Arizona,4.5,4.7,5.0,5.3,5.6,5.9,6.3,6.6,6.9,7.3,...,9.7,9.7,9.7,9.7,9.7,9.6,9.4,9.2,9.0,8.8
Arkansas,5.0,5.0,4.9,5.0,5.0,5.2,5.3,5.5,5.8,6.1,...,8.5,8.5,8.5,8.4,8.4,8.3,8.2,8.0,7.9,7.8


In [ ]:

t_df

In [33]:
for col in state_unemployment_df.columns[3:]:
    print(col , state_unemployment_df[col].astype(float).mean() df.iloc[col,lt].value)

SyntaxError: invalid syntax (<ipython-input-33-30d06be61b59>, line 2)

In [145]:
import pandas as pd
avg_file="raw_avg.csv"
avg_file_df=pd.read_csv(avg_file)
avg_file_df.head(10)

,State,avg
0,Alabama,9.216667
1,Alaska,7.470833
2,Arizona,8.979167
3,Arkansas,7.452083
4,California,10.591667
5,Colorado,7.308333
6,Connecticut,7.887500
7,Delaware,7.285417
8,District Of Columbia,8.820833
9,Florida,9.429167


In [146]:
avg_file_df.rename(columns={"State" : "name"},inplace=True)
avg_file_df   
    

,name,avg
0,Alabama,9.216667
1,Alaska,7.470833
2,Arizona,8.979167
3,Arkansas,7.452083
4,California,10.591667
5,Colorado,7.308333
6,Connecticut,7.887500
7,Delaware,7.285417
8,District Of Columbia,8.820833
9,Florida,9.429167


In [147]:
avg_file2_df=avg_file_df.T
avg_file2_df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
name,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District Of Columbia,Florida,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,​
avg,9.21667,7.47083,8.97917,7.45208,10.5917,7.30833,7.8875,7.28542,8.82083,9.42917,...,8.92083,7.07083,6.37083,5.70625,6.08958,8.44792,7.18958,7.475,5.41042,NaN


In [ ]:
avg_file_df[" avg"] = avg_file_df[" avg"].map("{:.2f}".format)

In [148]:
df = pd.read_csv("states.csv",delimiter="\t")
df.head()

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [149]:
statelo_df=df
statelo_df.head()

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [150]:
merge_df = pd.merge(statelo_df, avg_file_df, on="name")
merge_df.head(10)

,state,latitude,longitude,name,avg
0,AK,63.588753,-154.493062,Alaska,7.470833
1,AL,32.318231,-86.902298,Alabama,9.216667
2,AR,35.201050,-91.831833,Arkansas,7.452083
3,AZ,34.048928,-111.093731,Arizona,8.979167
4,CA,36.778261,-119.417932,California,10.591667
5,CO,39.550051,-105.782067,Colorado,7.308333
6,CT,41.603221,-73.087749,Connecticut,7.887500
7,DE,38.910832,-75.527670,Delaware,7.285417
8,FL,27.664827,-81.515754,Florida,9.429167
9,GA,32.157435,-82.907123,Georgia,9.231250


In [151]:
# df['a'].values.tolist()
# [1, 3, 5, 7, 4, 5, 6, 4, 7, 8, 9]
ratelist = avg_file_df[" avg"].values.tolist()
ratelist

[9.216666666666667,
 7.470833333333335,
 8.979166666666666,
 7.452083333333332,
 10.591666666666667,
 7.308333333333333,
 7.8874999999999975,
 7.285416666666664,
 8.820833333333336,
 9.429166666666664,
 9.23125,
 6.299999999999998,
 7.789583333333333,
 9.141666666666666,
 8.914583333333333,
 5.539583333333333,
 6.260416666666665,
 9.085416666666664,
 6.883333333333336,
 7.404166666666664,
 6.541666666666668,
 7.2958333333333325,
 11.154166666666667,
 6.772916666666667,
 9.10416666666667,
 8.374999999999998,
 6.5479166666666675,
 4.25,
 11.13125,
 5.333333333333335,
 8.316666666666665,
 6.902083333333335,
 7.666666666666668,
 9.447916666666664,
 3.6354166666666674,
 8.964583333333335,
 5.7,
 9.483333333333333,
 7.431249999999999,
 10.25208333333333,
 9.939583333333333,
 4.43125,
 8.920833333333334,
 7.07083333333333,
 6.370833333333334,
 5.706250000000002,
 6.089583333333333,
 8.447916666666666,
 7.189583333333334,
 7.474999999999998,
 5.410416666666664,
 nan]

In [152]:
avg_file2_df.columns = avg_file2_df.loc[" avg"].to_numpy()
avg_file2_df.head

<bound method NDFrame.head of      9.216667  7.470833  8.979167  7.452083    10.591667 7.308333   \
name   Alabama    Alaska   Arizona  Arkansas  California  Colorado   
 avg   9.21667   7.47083   8.97917   7.45208     10.5917   7.30833   

        7.887500  7.285417              8.820833  9.429167   ...  8.920833   \
name  Connecticut  Delaware  District Of Columbia   Florida  ...  Tennessee   
 avg       7.8875   7.28542               8.82083   9.42917  ...    8.92083   

     7.070833  6.370833  5.706250  6.089583    8.447917       7.189583   \
name     Texas      Utah   Vermont  Virginia  Washington  West Virginia   
 avg   7.07083   6.37083   5.70625   6.08958     8.44792        7.18958   

      7.475000  5.410417  NaN        
name  Wisconsin   Wyoming         ​  
 avg      7.475   5.41042       NaN  

[2 rows x 52 columns]>

In [153]:
# gmaps.configure(api_key=g_key)
# locations = cities_df[["Lat", "Lng"]]
# Humidity = cities_df["Humidity"]
# max_humidity = cities_df["Humidity"].max()
import gmaps
from test_key import g_key
import os

gmaps.configure(api_key=g_key)
locations = merge_df[["latitude","longitude"]]
unemploymentrate= merge_df[" avg"]
max_unemployment=merge_df[" avg"].max()

In [154]:
# fig = gmaps.figure()
# heatmap = gmaps.heatmap_layer(locations, weights=Humidity, max_intensity=max_humidity)
# fig.add_layer(heatmap)

# fig
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = unemploymentrate, max_intensity = max_unemployment)
fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

In [71]:
fig2= gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = unemploymentrate, max_intensity = max_unemployment, point_radius = 30)
fig2.add_layer(heatmap)

fig2

Figure(layout=FigureLayout(height='420px'))

In [157]:
rate_df=pd.read_csv("raw_avg.csv")
rate_df.head()

,State,avg
0,Alabama,9.216667
1,Alaska,7.470833
2,Arizona,8.979167
3,Arkansas,7.452083
4,California,10.591667


In [ ]:
# names = ['Atucha', 'Embalse', 'Armenia', 'BR', 'Doel']
# >>> symbol_layer = gmaps.symbol_layer(locations, hover_text=names)
 
# avgrate= ratelist
# symbol_layer = gmaps.symbol_layer(locations, hover_text= )
# fig.add_layer(symbol_layer)


In [168]:
rates = merge_df[" avg"]

rate_info=[f"Average Unmployment Rate Avg:{rate}%" for rate in rates]
rate_info

['Average Unmployment Rate Avg:7.470833333333335%',
 'Average Unmployment Rate Avg:9.216666666666667%',
 'Average Unmployment Rate Avg:7.452083333333332%',
 'Average Unmployment Rate Avg:8.979166666666666%',
 'Average Unmployment Rate Avg:10.591666666666667%',
 'Average Unmployment Rate Avg:7.308333333333333%',
 'Average Unmployment Rate Avg:7.8874999999999975%',
 'Average Unmployment Rate Avg:7.285416666666664%',
 'Average Unmployment Rate Avg:9.429166666666664%',
 'Average Unmployment Rate Avg:9.23125%',
 'Average Unmployment Rate Avg:6.299999999999998%',
 'Average Unmployment Rate Avg:5.539583333333333%',
 'Average Unmployment Rate Avg:7.789583333333333%',
 'Average Unmployment Rate Avg:9.141666666666666%',
 'Average Unmployment Rate Avg:8.914583333333333%',
 'Average Unmployment Rate Avg:6.260416666666665%',
 'Average Unmployment Rate Avg:9.085416666666664%',
 'Average Unmployment Rate Avg:6.883333333333336%',
 'Average Unmployment Rate Avg:7.2958333333333325%',
 'Average Unmployme

In [160]:
rate_df

,State,avg
0,Alabama,9.216667
1,Alaska,7.470833
2,Arizona,8.979167
3,Arkansas,7.452083
4,California,10.591667
5,Colorado,7.308333
6,Connecticut,7.887500
7,Delaware,7.285417
8,District Of Columbia,8.820833
9,Florida,9.429167


In [169]:
print(len(rate_info))
print(len(locations))
print(len(merge_df))

50
50
50


In [166]:
merge_df.head()

,state,latitude,longitude,name,avg
0,AK,63.588753,-154.493062,Alaska,7.470833
1,AL,32.318231,-86.902298,Alabama,9.216667
2,AR,35.201050,-91.831833,Arkansas,7.452083
3,AZ,34.048928,-111.093731,Arizona,8.979167
4,CA,36.778261,-119.417932,California,10.591667


In [163]:
locations.columns

Index(['latitude', 'longitude'], dtype='object')

In [182]:
fig3= gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = unemploymentrate, max_intensity = max_unemployment, point_radius = 30)
fig3.add_layer(heatmap)

#demo_text = [f"{r['state']}: {r['rate']}" for i, r in demo_df.iterrows()]
hover_text=[f"{r['name']}:{r[' avg']}" for i , r in merge_df.iterrows()]

symbol_layer = gmaps.symbol_layer(merge_df[["latitude", "longitude"]], fill_color="blue",stroke_color="blue",scale= 2,
                                info_box_content= rate_info, hover_text=hover_text)

fig3.add_layer(symbol_layer)
fig3 

Figure(layout=FigureLayout(height='420px'))

In [178]:
# demo_states = ["A", "B", "C"]
# demo_rates = [1, 2, 3]
# demo_df = pd.DataFrame({"state": demo_states, "rate": demo_rates})
# demo_text = [f"{r['state']}: {r['rate']}" for i, r in demo_df.iterrows()]
# demo_text

['A: 1', 'B: 2', 'C: 3']

In [ ]:
#[f"Average Unmployment Rate Avg:{Rate}%" for Rate in rate_df["Rate"]

In [ ]:
import time
from scipy.stats import linregress
from statepy import statepy

## Unemployment level

In [ ]:
# Pull unemploymend level data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LNS13000000", "LNS13023621", "LNS13023653", "LNS13025699", "LNS13023705", "LNS13023557", "LNS13023569"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data4 = json.loads(p.text)

json_data4

In [ ]:
years = []
months = []
date = []

for data_point in np.arange(len(json_data4["Results"]["series"][0]["data"])):
        years.append(json_data4["Results"]["series"][0]["data"][data_point]["year"])
        months.append(json_data4["Results"]["series"][0]["data"][data_point]["periodName"])
        date.append(f"{json_data['Results']['series'][0]['data'][data_point]['year']}-{json_data['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [ ]:
# Create DataFrame

layoffs_df = pd.DataFrame({"Date": date,
                           "years": years,
                           "months": months,
                           "LNS13000000": "",
                           "LNS13023621": "",
                           "LNS13023653": "",
                           "LNS13025699": "",
                           "LNS13023705": "",
                           "LNS13023557": "",
                           "LNS13023569": ""})
layoffs_df.head()

In [ ]:
# Fills in values

for series4 in np.arange(len(json_data4["Results"]["series"])):
    for data_point4 in np.arange(len(json_data4["Results"]["series"][series4]["data"])):
        layoffs_df.loc[data_point4, json_data4["Results"]["series"][series4]["seriesID"]] = json_data4["Results"]["series"][series4]["data"][data_point4]["value"]


In [ ]:
# Rename column labels

layoffs_df.rename(columns={"LNS13000000": "Unemployment level, 16yo and over, thousands",
                           "LNS13023621": "Job losers, 16yo and over, thousands",
                           "LNS13023653": "Job losers on layoff, 16yo and over, thousands",
                           "LNS13025699": "Job losers not on layoff, 16yo and over, thousands",
                           "LNS13023705": "Job leavers (quit), 16yo and over, thousands",
                           "LNS13023557": "Reentrants to labor force, 16yo and over, thousands",
                           "LNS13023569": "New entrants to labor force, 16yo and over, thousands"},
                 inplace=True)

layoffs_df.sort_values(by="Date", ascending=True, inplace=True)

layoffs_df.to_csv("Unemployment_Levels.csv", index=False)

layoffs_df

## COVID cases

In [ ]:
url = "https://covidtracking.com/api/us/daily"

response = requests.get(url).json()

print(json.dumps(response, indent=True))

In [ ]:
dates = []
cum_positives = []
cum_deaths = []
inc_positives = []
inc_deaths = []

In [ ]:
for day in np.arange(len(response)):
    dates.append(response[day]["date"])
    cum_positives.append(response[day]["positive"])
    cum_deaths.append(response[day]["death"])
    inc_positives.append(response[day]["positiveIncrease"])
    inc_deaths.append(response[day]["deathIncrease"])

In [ ]:
sick_df = pd.DataFrame({"Date" : dates,
                       "Total Cases" : cum_positives,
                       "Total Deaths" : cum_deaths,
                       "Daily increase in Cases" : inc_positives,
                       "Daily increase in Deaths" : inc_deaths})

sick_df.loc[len(inc_positives)-1, "Daily increase in Deaths"] = sick_df.loc[len(cum_positives)-1, "Total Deaths"]
sick_df.loc[len(inc_deaths)-1, "Daily increase in Cases"] = sick_df.loc[len(cum_deaths)-1, "Total Cases"]

sick_df["Date"] = sick_df["Date"].astype(str)

In [ ]:
for x in np.arange(len(dates)):
    sick_df.loc[x,"Date"] = f"{sick_df.loc[x,'Date'][0:4]}-{sick_df.loc[x,'Date'][4:6]}-{sick_df.loc[x,'Date'][6:]}"

sick_df.sort_values(by="Date", ascending=True, inplace=True)

In [ ]:
sick_df.to_csv("COVID_cases.csv", index=False)
sick_df

## Economic Indicators

In [ ]:
file_path1 = "Resources/2020_GSPC.csv"
file_path2 = "Resources/2020_DJI.csv"
file_path3 = "Resources/2020_IXIC.csv"
file_path4 = "Resources/2020_RUT.csv"

SP500_df = pd.read_csv(file_path1)
Dow_df = pd.read_csv(file_path2)
NASDAQ_df = pd.read_csv(file_path3)
Russell2000_df = pd.read_csv(file_path4)

In [ ]:
SP500_df = SP500_df[["Date", "Close", "Volume"]]
Dow_df = Dow_df[["Date", "Close", "Volume"]]
NASDAQ_df = NASDAQ_df[["Date", "Close", "Volume"]]
Russell2000_df = Russell2000_df[["Date", "Close", "Volume"]]

In [ ]:
eco_ind_df = pd.merge(SP500_df, Dow_df, on="Date", how="outer", suffixes=["_SP500", "_Dow_Jones_Industrial_Average"])
eco_ind_2_df = pd.merge(NASDAQ_df, Russell2000_df, on="Date", how="outer", suffixes=["_NASDAQ", "_Russell2000"])
eco_ind_df = pd.merge(eco_ind_df, eco_ind_2_df, on="Date", how="outer")

eco_ind_df.sort_values(by="Date", ascending=True, inplace=True)

eco_ind_df

In [ ]:
eco_ind_df.to_csv("Eco_Indicators.csv", index=False)

In [ ]:
COVID_econ_df = pd.merge(sick_df, eco_ind_df, on="Date", how="inner")
COVID_econ_df = COVID_econ_df.drop(["Daily increase in Cases", "Daily increase in Deaths"], axis=1)
COVID_econ_df